In [ ]:
!pip install requests beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import re

def extract_text(element):
    return element.get_text(separator=' ', strip=True)

def scrape(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        main_wrap_div = soup.find("div", class_="main-wrap")

        if main_wrap_div:
            sub_div_blocks = main_wrap_div.find_all("div", class_="column half b-col")

            for sub_div in sub_div_blocks:
                link = sub_div.find("a", class_="image-link")
                if link and 'href' in link.attrs:
                    article_url = link['href']
                    article_response = requests.get(article_url, headers=headers)

                    if article_response.status_code == 200:
                        article_soup = BeautifulSoup(article_response.content, "html.parser")
                        # Extracting claim, fact, and fact-check based on the number of <p> and <strong> tags
                        blockquote = article_soup.select_one('.wp-block-quote')
                        if blockquote:
                          p_tags = blockquote.find_all('p')
                          strong_tags = blockquote.find_all('strong')

                          if len(p_tags) == 1 and len(strong_tags) >= 3:
                            # Extracting the content based on the specified tags
                              fact_all = article_soup.select_one('.wp-block-quote > p').text.strip()
                              if "ఫాక్ట్(నిజం):" in fact_all:
                                claim_ = fact_all[ :fact_all.find("ఫాక్ట్(నిజం):")]
                                fact_ = fact_all[fact_all.find("ఫాక్ట్(నిజం):"): ]
                              elif "ఫాక్ట్ (నిజం):" in fact_all:
                                claim_ = fact_all[ :fact_all.find("ఫాక్ట్ (నిజం):")]
                                fact_ = fact_all[fact_all.find("ఫాక్ట్ (నిజం):"): ]
                              elif "ఫాక్ట్:" in fact_all:
                                claim_ = fact_all[ :fact_all.find("ఫాక్ట్:")]
                                fact_ = fact_all[fact_all.find("ఫాక్ట్:"): ]
                              elif "ఫాక్ట్(నిజం) :" in fact_all:
                                claim_ = fact_all[ :fact_all.find("ఫాక్ట్(నిజం) :")]
                                fact_ = fact_all[fact_all.find("ఫాక్ట్(నిజం) :"): ]
                              elif "ఫాక్ట్ (నిజం) :" in fact_all:
                                claim_ = fact_all[ :fact_all.find("ఫాక్ట్ (నిజం) :")]
                                fact_ = fact_all[fact_all.find("ఫాక్ట్ (నిజం) :"): ]
                              elif "ఫాక్ట్ :" in fact_all:
                                claim_ = fact_all[ :fact_all.find("ఫాక్ట్ :")]
                                fact_ = fact_all[fact_all.find("ఫాక్ట్ :"): ]
                              elif "ఫ్యాక్ట్ (నిజం):" in fact_all:
                                claim_ = fact_all[ :fact_all.find("ఫ్యాక్ట్ (నిజం):")]
                                fact_ = fact_all[fact_all.find("ఫ్యాక్ట్ (నిజం):"): ]
                              elif "ఫ్యాక్ట్ (నిజం) :" in fact_all:
                                claim_ = fact_all[ :fact_all.find("ఫ్యాక్ట్ (నిజం) : ")]
                                fact_ = fact_all[fact_all.find("ఫ్యాక్ట్ (నిజం) : "): ]
                              else:
                                continue
                              rest_, claim = claim_.split(':',1)
                              rest, fact = fact_.split(':',1)
                              # claim = re.sub(r'^క్లెయిమ్:', '', claim_).lstrip()
                              # claim = re.sub(r'^క్లెయిమ్ :', '', claim_).lstrip()
                              # prefix_pattern = r'^[ఫాక్ట్\s(నిజం):\s]*'
                              # fact = re.sub(prefix_pattern, '', fact_).lstrip()
                              fact_check = article_soup.select_one('.wp-block-quote > p > strong:nth-of-type(3)').text.strip()

                          elif len(p_tags) == 2:
                              claim_ = article_soup.select_one('.wp-block-quote > p:nth-of-type(1)').text.strip()
                              claim = claim_.split(':',1)[1]
                              fact_ = article_soup.select_one('.wp-block-quote > p:nth-of-type(2)').text.strip()
                              fact = fact_.split(':',1)[1]

                            #   # claim = article_soup.select_one('.wp-block-quote > p:nth-of-type(1)').text.strip()
                            #   claim_ = article_soup.select_one('.wp-block-quote > p:nth-of-type(1)').text.strip()
                            #   claim = re.sub(r'^క్లెయిమ్:', '', claim_).lstrip()
                            #   claim = re.sub(r'^క్లెయిమ్ :', '', claim_).lstrip()
                            #   fact_ = article_soup.select_one('.wp-block-quote > p:nth-of-type(2)').text.strip()
                            #   prefix_pattern = r'^[ఫాక్ట్\s(నిజం):\s]*'
                            #   fact = re.sub(prefix_pattern, '', fact_).lstrip()

                              if article_soup.select_one('.wp-block-quote > p:nth-of-type(2) > strong:nth-of-type(2)'):
                                fact_check = article_soup.select_one('.wp-block-quote > p:nth-of-type(2) > strong:nth-of-type(2)').text.strip()
                              else:
                                if article_soup.select_one('.wp-block-quote > p:nth-of-type(2)> em >strong'):
                                  fact_check = article_soup.select_one('.wp-block-quote > p:nth-of-type(2)> em >strong').text.strip()
                                else:
                                  continue
                              # claim = extract_text(p_tags[0])
                              # fact = extract_text(p_tags[1])
                              # fact_check = ''
                          else:
                              continue

                          reviewer = article_soup.select_one('.post-meta.cf .author.url.fn').text.strip()
                          date_time = article_soup.select_one('.post-meta.cf .posted-on .dtreviewed time')['datetime']
                          date, time_str = date_time.split('T')
                          time,_ =time_str.split('+')

                          # Prepare the data for CSV
                          data = {
                              'Claim': claim,
                              'Fact': fact,
                              'Fact-check': fact_check,
                              'Reviewer': reviewer,
                              'Date(YYYY-MM-DD)': date,
                              'Time' : time
                          }

                          # Write to CSV
                          with open('output.csv', mode='a', newline='', encoding='utf-8') as file:
                              writer = csv.DictWriter(file, fieldnames=data.keys())
                              file.seek(0, 2)  # Move to the end of the file
                              if file.tell() == 0:
                                  writer.writeheader()
                              writer.writerow(data)

                          print(f'Data from {article_url} written to output.csv.')
                        else:
                          continue
                    else:
                        print(f"Failed to retrieve the article at {article_url}. Status code: {article_response.status_code}")
                else:
                    print("No valid link found in the sub div.")
        else:
            print("No valid sub div blocks found.")
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


# url = "https://factly.in/category/telugu/"
# scrape(url)

for i in range(1, 10):
    url = f"https://factly.in/category/telugu/page{i}"
    scrape(url)

print("")
print("completed")


Data from https://factly.in/telugu-this-photo-of-vs-achuthanandan-with-a-tilak-on-his-forehead-is-a-morphed-one/ written to output.csv.
Data from https://factly.in/telugu-this-disha-news-clipping-reporting-about-revanth-reddys-secret-meeting-with-chandrababu-naidu-is-fake/ written to output.csv.
Data from https://factly.in/telugu-this-letter-in-the-name-of-chandrababu-naidu-to-kammas-in-telangana-is-fake/ written to output.csv.
Data from https://factly.in/telugu-israel-police-did-not-expel-this-woman-in-the-video-back-to-gaza-for-posting-the-whatsapp-status-in-support-of-hamas/ written to output.csv.
Data from https://factly.in/telugu-a-digitally-manipulated-video-is-being-shared-on-social-media-to-claim-that-ktr-got-heckled-by-congress-supporters-during-his-speech-in-kodangal/ written to output.csv.
Data from https://factly.in/telugu-the-person-who-is-spreading-the-message-about-exercise-of-right-to-vote-in-karimnagar-district-is-not-an-army-jawan/ written to output.csv.
Data from htt